In [5]:
import tensorflow as tf
import numpy as np

In [3]:
batch_size = 4
n_classes = 5

In [4]:
y_pred = tf.constant(np.random.randn(batch_size,n_classes))
y_true = tf.constant(np.random.randint(0,n_classes,size=(batch_size,1)))
tf.Session().run(y_pred), tf.Session().run(y_true)

(array([[ 0.24367752,  1.70646145,  0.94334228, -0.07368144, -0.28523229],
        [ 0.61927867,  0.87952458,  0.20126887,  1.43617454,  0.33950709],
        [ 0.81890233, -0.25468515, -0.28454964, -0.86632344,  3.29608533],
        [ 0.40979566, -0.51896338,  0.65311239,  2.26200326,  1.12645674]]),
 array([[0],
        [4],
        [1],
        [3]]))

In [28]:
def softmax_neg(logits):
    mask = 1 - tf.eye(batch_size, batch_size, dtype=tf.float64)
    neg_scores = mask * logits
    diff = neg_scores - tf.reduce_max(neg_scores, axis=1)
    exp = tf.math.exp(diff) * mask
    softmaxed = exp / tf.reduce_sum(exp, axis=1)
    return softmaxed

In [7]:
y_true = tf.reshape(y_true,[-1])
gather = tf.gather(y_pred,y_true, axis=1) # get positive and negative scores
y_softmax = softmax_neg(gather)
diag = tf.diag_part(gather) # positive samples
diag_exp = tf.expand_dims(diag, axis=0) # expand dim to transpose
trans = tf.transpose(diag_exp) 
diff = trans - gather # diference between positive and all
sig = tf.nn.sigmoid(diff)
loss = tf.reduce_mean(-tf.log(sig))
tf.Session().run(loss)

0.9090760956347723

In [29]:
soft_neg = softmax_neg(gather)
tf.Session().run(soft_neg)

array([[0.00000000e+00, 9.99279502e-02, 2.92638442e-02, 2.92627595e-01],
       [4.92953421e-01, 0.00000000e+00, 1.27996253e-02, 1.32445557e+00],
       [6.01868119e-01, 3.58949588e+00, 0.00000000e+00, 1.32457095e-01],
       [3.99786911e-01, 4.09992331e-01, 3.16112486e-03, 0.00000000e+00]])

In [41]:
def bpr_max(y_true, y_pred):
    y_true = tf.reshape(y_true,[-1])
    gather = tf.gather(y_pred,y_true, axis=1) # get positive and negative scores, gather=yhat
    y_softmax = softmax_neg(gather)
    diag = tf.diag_part(gather) # positive samples
    diag_exp = tf.expand_dims(diag, axis=0) # expand dim to transpose
    trans = tf.transpose(diag_exp) 
    diff = trans - gather
    sig = tf.nn.sigmoid(diff) * y_softmax
    reg = 0.001 * tf.reduce_sum(((gather**2)*y_softmax), axis=1)
    loss = tf.reduce_mean(-tf.log(sig + 1e-24) + reg)
    return loss
    
    

In [42]:
tf.Session().run(bpr_max(y_true,y_pred))

15.853622751566789